In [ ]:
# 可刪掉的block
"/Users/cfh00892302/Desktop/myWorkspace/Chinese-Verdict-NLP/dev/label_output_new/merged_data.json"

1. 把資料合併 #暫時不用做這步驟

In [2]:
from information_extraction.utils.base_utils import merge_json
all_json_file_locate_folder = "./dev/label_output_raw"
merged_json_output_folder = "./dev/label_output_new"
merge_json(all_json_file_locate_folder, merged_json_output_folder)

[2023-05-26 14:19:18,522] [    INFO] - Find the following json file: ['個理_文本分類_119.json', '商險_文本分類_25.json'].
[2023-05-26 14:19:18,523] [    INFO] - Merging the file 個理_文本分類_119.json...
[2023-05-26 14:19:18,536] [    INFO] - Length of 個理_文本分類_119.json is 119.
[2023-05-26 14:19:18,538] [    INFO] - Merging the file 商險_文本分類_25.json...
[2023-05-26 14:19:18,543] [    INFO] - Length of 商險_文本分類_25.json is 25.
[2023-05-26 14:19:18,561] [    INFO] - Successful merge all file (len=144) to ./dev/label_output_new/merged_data.json.


In [6]:
file_name = "./dev/label_output_new/merged_data.json"

2. 檢查資料正確性

In [11]:
from information_extraction.utils.base_utils import is_repeat_content_exist
result = is_repeat_content_exist(file_name)
if not result['is_repeat']:
    print("OK! No repeat.")
else:
    print("Duplicate cases happened!!!")
    

[2023-05-26 14:21:56,809] [    INFO] - All data: 144. Unique data: 144. Repeat data: 0.


OK! No repeat.


3. Split into train/eval/test

In [ ]:
!python information_extraction/split_labelstudio.py \
    --labelstudio_file ./dev/label_output_new/merged_data.json \
    --save_dir information_extraction/first_stage_data/ \
    --split_ratio 0.8 0.1 0.1 